# Tutorial demonstrating verification of v1 precip against jra55

#### Import pyLatte package

In [96]:
from pylatte import utils
from pylatte import skill

#### Currently, the following packages are required to load the data - this process will be replaced by the CAFE cookbook

In [97]:
import numpy as np
import pandas as pd
import xarray as xr

#### Import some plotting packages and widgets

In [98]:
import matplotlib.pyplot as plt
import warnings    
warnings.filterwarnings("ignore")

# Jupyter specific -----
from ipywidgets import FloatProgress
%matplotlib inline

# A note about the methodology of pyLatte
The pyLatte package is constructed around the xarray Python package. This is particularly useful for verification computations which require large numbers of samples (different model runs) to converge. 

The approach here is to generate very large xarray objects that reference all data required for the verification, but do not store the data in memory. Operations are performed on these xarray objects out-of-memory. When it is necessary to perform a compute (e.g. to produce a plot), this is distributed over multiple processors using the dask Python package.

# Initialise dask (currently not working on vm31)

In [99]:
# import dask
# import distributed
# client = distributed.Client(local_dir='/tmp/squ027-dask-worker-space', n_workers=4)
# client

# Construct xarray objects for forecasts and observations
(The CAFE cookbook will replace these code blocks)

In [100]:
# Location of forecast data -----
fcst_folder = '/OSM/CBR/OA_DCFP/data/model_output/CAFE/forecasts/v1/'
fcst_filename = 'atmos_daily*'
fcst_variable = 'precip'

# Location of observation data -----
obsv_folder = '/OSM/CBR/OA_DCFP/data/observations/jra55/isobaric/061_tprat/cat/'
obsv_filename = 'jra.55.tprat.000.1958010100_2016123121.nc'
obsv_variable = 'TPRAT_GDS0_SFC_ave3h'

In [101]:
# Initial dates to include (takes approximately 1 min 30 sec per date) -----
init_dates = pd.date_range('2003-1','2003-5' , freq='1MS')

# Ensembles to include -----
ensembles = range(1,12)

# Forecast length -----
FCST_LENGTH = 2 # years

In [102]:
# Resampling details -----
resample_freq = 'MS'

### Construct forecasts xarray object
Note, dask has a known bug that manifests when trying to concatentate data containing timedelta64 arrays (see https://github.com/pydata/xarray/issues/1952 for further details). For example, try to concatenate the following two Datasets:

`In : path = '/OSM/CBR/OA_DCFP/data/model_output/CAFE/forecasts/v1/yr2002/mn7/'`

`In : ens5 = xr.open_mfdataset(path + 'OUTPUT.5/atmos_daily*.nc', autoclose=True)`

`In : ens6 = xr.open_mfdataset(path + 'OUTPUT.6/atmos_daily*.nc', autoclose=True)`

`In : xr.concat([ens5, ens6],'ensemble')`

`Out : TypeError: invalid type promotion`

The error here is actually caused by the variables `average_DT` and `time_bounds`, which are timedelta64 arrays. However, I still do not fully unstand the bug: concatenation of `ens4` and `ens5`, for example, works fine, even though `ens4` also contains the timedelta64 variables `average_DT` and `time_bounds`. Regardless, because of this bug, it is not possible currently to create an xarray Dataset object containing all model variables. Instead, only the variable of interest (i.e. `fcst_variable` and `obsv_variable`) are retained in the concatenated xarray object.

In [103]:
# Instantiate progress bar -----
f = FloatProgress(min=0, max=len(init_dates)*len(ensembles), description='Loading...') 
display(f)

# Loop over initial dates -----
fcst_list = []
for init_date in init_dates:
    year = init_date.year
    month = init_date.month
    
    # Loop over ensembles -----
    ens_list = []
    for ensemble in ensembles:
        # Signal to increment the progress bar -----
        f.value += 1 
        
        # Stack ensembles into a list -----
        path = fcst_folder + '/yr' + str(year) + '/mn' + str(month) + \
               '/OUTPUT.' + str(ensemble) + '/' + fcst_filename + '.nc'
        dataset = xr.open_mfdataset(path, autoclose=True)[fcst_variable]
        ens_list.append(dataset.resample(time=resample_freq) \
                               .sum(dim='time'))
        
    # Concatenate ensembles -----
    ens_object = xr.concat(ens_list, dim='ensemble')
    ens_object['ensemble'] = ensembles
    
    # Stack concatenated ensembles into a list for each initial date -----                       
    fcst_list.append(utils.datetime_to_leadtime(ens_object))

# Keep track of the lead time for each initialization -----
n_lead_time = [len(x.lead_time) for x in fcst_list]

# Concatenate initial dates -----
da_fcst = xr.concat(fcst_list, dim='init_date')

# Rechunk for chunksizes of at least 1,000,000 elements -----
da_fcst = utils.prune(da_fcst.chunk(chunks={'ensemble' : len(da_fcst.ensemble), 
                                            'lead_time' : len(da_fcst.lead_time)}).squeeze())

FloatProgress(value=0.0, description='Loading...', max=55.0)

#### Truncate the forecast lead times at 2 years
The January and July forecasts are run for 5 years rather than 2 years. The xarray concatenation above can deal with this, but fills the shorter forecasts with nans for lead times longer than 2 years. Let's get rid of some of these nans by truncating the forecasts at the lead time corresponding to the longest 2 year forecast.

In [105]:
max_increments = FCST_LENGTH * 12
n_trunc = max([i for i in n_lead_time if i <= max_increments])
da_fcst = da_fcst.isel(lead_time=range(n_trunc))

### Construct observations xarray object

In [106]:
# Instantiate progress bar -----
f = FloatProgress(min=0, max=1, description='Loading...') 
display(f)

# JRA temperature fields are only save in a time-concatenated form -----
path = obsv_folder + obsv_filename
dataset = xr.open_mfdataset(path, autoclose=True)[obsv_variable]
da_obsv = dataset.rename(fcst_variable) \
                 .rename({'initial_time0_hours' : 'time', 'g0_lon_3' : 'lon', 'g0_lat_2' : 'lat'}) \
                 .resample(time=resample_freq) \
                 .sum(dim='time')

# Stack by initial date to match forecast structure -----
da_obsv = utils.stack_by_init_date(da_obsv,da_fcst.init_date.values,n_trunc)
f.value += 1

# Average over forecast dimension if it is exists -----
if 'forecast_time1' in da_obsv.coords:
    da_obsv = da_obsv.mean(dim='forecast_time1')

# Rechunk for chunksizes of at least 1,000,000 elements -----
da_obsv = utils.prune(da_obsv.chunk(chunks={'init_date' : len(da_obsv.init_date)}).squeeze())

FloatProgress(value=0.0, description='Loading...', max=1.0)

# Let's look at average monthly rainfall over Tasmania

##### Extract forecast and observation over region
Note, we `compute()` the xarray objects here to save time later on. Once dask is working, it will probably be most sensible to leave the objects uncomputed

In [107]:
with utils.timer():
    # Region of interest -----
    region = (-44.0, -40.0, 144.0 , 148.0) # (lat_min,lat_max,lon_min,lon_max)

    unit_conv = 60 * 60 * 24 / 998.2 * 1000
    da_fcst = utils.calc_boxavg_latlon(da_fcst * unit_conv, region).compute()

    da_obsv = utils.calc_boxavg_latlon(da_obsv, region).compute()

   Elapsed: 48.50764465332031 sec


##### Load climatology data
Various climatologies are/will be accessable using `utils.load_climatology()`. Here we use a climatology computed over the full 55 year jra reanalysis

In [108]:
jra_clim = utils.load_climatology('jra_1958-2016', 'precip', freq='MS')

da_jra_clim = utils.calc_boxavg_latlon(jra_clim, region).compute()

##### Compute anomaly data
Recall that the forecast and observation data are saved as functions of lead time and initial date. The function `utils.anomalize()` computes anomalies given data and a climatology which each have a datetime dimension `time`. Thus it is necessary to first convert from the lead time/initial date format to a datetime format, then compute the anomaly, the convert back to the lead time/initial date format. The functions `utils.datetime_to_leadtime()` and `utils.leadtime_to_datetime()` enable these types of operations

In [109]:
anomalize = lambda data, clim: utils.datetime_to_leadtime(
                                   utils.anomalize(
                                       utils.leadtime_to_datetime(data),clim))

In [110]:
da_fcst_anom = da_fcst.groupby('init_date').apply(anomalize, clim=da_jra_clim)

da_obsv_anom = da_obsv.groupby('init_date').apply(anomalize, clim=da_jra_clim)

##### Compute persistence data
This requires repeating the data at the first lead time over all lead times. `utils.repeat_data()` allows us to do this

In [111]:
da_pers = utils.repeat_data(da_obsv,'lead_time')

##### Compute climatogoloy in lead time/inital date format
This is really just for convenience below

In [112]:
da_clim = (0 * da_obsv).groupby('init_date').apply(anomalize, clim=-da_jra_clim)

## Edit metrics to use better loop structure

In [140]:
def compute_Brier_score_old(cmp_likelihood, ref_logical, indep_dims, cmp_prob=None):
    """ 
    Computes the Brier score(s) of an event given the comparison likelihood and reference logical 
    event data. When comparison probability bins are also provided, also computes the reliability, 
    resolution and uncertainty components of the Brier score
    """
    
    if isinstance(indep_dims, str):
        indep_dims = [indep_dims]
    
    ref_binary = ref_logical.copy()*1

    # Calculate total Brier score -----
    N = 1
    if indep_dims == None:
        Brier = (1 / N) * ((cmp_likelihood - ref_binary) ** 2).sum(dim=indep_dims).rename('Brier_score')
    else: 
        for indep_dim in indep_dims:
            N = N * len(cmp_likelihood[indep_dim])
        Brier = (1 / N) * ((cmp_likelihood - ref_binary) ** 2).rename('Brier_score')
        
    # Calculate components
    if cmp_prob is not None:

        # Initialise probability bins -----
        cmp_prob_edges = utils.get_bin_edges(cmp_prob)

        # Initialise mean_cmp_prob array -----
        mean_cmp_likelihood = cmp_likelihood.copy()
        
        # Logical of comparisons that fall within first probability bin -----
        cmp_in_bin = (cmp_likelihood >= cmp_prob_edges[0]) & \
                      (cmp_likelihood < cmp_prob_edges[1])

        # Compute mean comparison probability -----
        if indep_dims == None:
            mean_cmp_likelihood = mean_cmp_likelihood.where(~cmp_in_bin) \
                                   .fillna(mean_cmp_likelihood.where(cmp_in_bin))
        else:
            mean_cmp_likelihood = mean_cmp_likelihood.where(~cmp_in_bin) \
                                   .fillna(mean_cmp_likelihood.where(cmp_in_bin).mean(dim=indep_dims))
                
        # Mean comparison probability within first probability bin -----
        if indep_dims == None:
            mean_cmp_prob = cmp_likelihood.where(cmp_in_bin,np.nan)
        else:
            mean_cmp_prob = cmp_likelihood.where(cmp_in_bin,np.nan).mean(dim=indep_dims)
        mean_cmp_prob.coords['comparison_probability'] = cmp_prob[0]
        mean_cmp_prob = mean_cmp_prob.expand_dims('comparison_probability')

        # Number of comparisons that fall within probability bin -----
        if indep_dims == None:
            cmp_number = cmp_in_bin
        else:
            cmp_number = cmp_in_bin.sum(dim=indep_dims)
        cmp_number.coords['comparison_probability'] = cmp_prob[0]
        cmp_number = cmp_number.expand_dims('comparison_probability')

        # Number of reference occurences where comparison likelihood is within probability bin -----
        if indep_dims == None:
            ref_occur = ((cmp_in_bin == True) & (ref_logical == True))
        else:
            ref_occur = ((cmp_in_bin == True) & (ref_logical == True)).sum(dim=indep_dims)
        ref_occur.coords['comparison_probability'] = cmp_prob[0]
        ref_occur = ref_occur.expand_dims('comparison_probability')

        # Loop over probability bins -----
        for idx in range(1,len(cmp_prob_edges)-1):
            # Logical of comparisons that fall within probability bin -----
            del cmp_in_bin
            cmp_in_bin = (cmp_likelihood >= cmp_prob_edges[idx]) & \
                          (cmp_likelihood < cmp_prob_edges[idx+1])
            
            if indep_dims == None:
                mean_cmp_likelihood = mean_cmp_likelihood.where(~cmp_in_bin) \
                                   .fillna(mean_cmp_likelihood.where(cmp_in_bin))
            else:
                mean_cmp_likelihood = mean_cmp_likelihood.where(~cmp_in_bin) \
                                   .fillna(mean_cmp_likelihood.where(cmp_in_bin).mean(dim=indep_dims))
            cmp_in_bin.coords['comparison_probability'] = cmp_prob[idx]
            
            # Mean comparison probability within current probability bin -----
            if indep_dims == None:
                mean_cmp_prob_temp = cmp_likelihood.where(cmp_in_bin,np.nan)
            else:
                mean_cmp_prob_temp = cmp_likelihood.where(cmp_in_bin,np.nan).mean(dim=indep_dims)
            mean_cmp_prob_temp.coords['comparison_probability'] = cmp_prob[idx]
            mean_cmp_prob = xr.concat([mean_cmp_prob, mean_cmp_prob_temp],
                                       dim='comparison_probability')

            # Number of comparisons that fall within probability bin -----
            if indep_dims == None:
                cmp_number = xr.concat([cmp_number, cmp_in_bin],dim='comparison_probability')
            else:
                cmp_number = xr.concat([cmp_number, cmp_in_bin.sum(dim=indep_dims)], \
                                        dim='comparison_probability')
        
            # Number of reference occurences where comparison likelihood is within probability bin -----
            if indep_dims == None:
                ref_occur = xr.concat([ref_occur, ((cmp_in_bin == True) \
                                                     & (ref_logical == True))],dim='comparison_probability')
            else:
                ref_occur = xr.concat([ref_occur, ((cmp_in_bin == True) \
                                                     & (ref_logical == True)) \
                                          .sum(dim=indep_dims)],dim='comparison_probability')

        # Compute Brier components -----
        base_rate = ref_occur / cmp_number
        Brier_reliability = (1 / N) * (cmp_number*(mean_cmp_prob - base_rate) ** 2) \
                                       .sum(dim='comparison_probability',skipna=True)
        if indep_dims == None:
            sample_clim = ref_binary
        else:
            sample_clim = ref_binary.mean(dim=indep_dims)
        Brier_resolution = (1 / N) * (cmp_number*(base_rate - sample_clim) ** 2) \
                                      .sum(dim='comparison_probability',skipna=True)
        Brier_uncertainty = sample_clim * (1 - sample_clim)
        
        # When a binned approach is used, compute total Brier using binned probabilities -----
        # (This way Brier_total = Brier_reliability - Brier_resolution + Brier_uncertainty)
        if indep_dims == None:
            Brier_total = (1 / N) * ((mean_cmp_likelihood - ref_binary) ** 2)
        else:
            Brier_total = (1 / N) * ((mean_cmp_likelihood - ref_binary) ** 2).sum(dim=indep_dims)
        
        # Package in dataset -----
        Brier = Brier_total.to_dataset(name='Brier_total')
        Brier.Brier_total.attrs['name'] = 'total Brier score'
        Brier['Brier_reliability'] = Brier_reliability
        Brier.Brier_reliability.attrs['name'] = 'reliability component of Brier score'
        Brier['Brier_resolution'] = Brier_resolution
        Brier.Brier_resolution.attrs['name'] = 'resolution component of Brier score'
        Brier['Brier_uncertainty'] = Brier_uncertainty
        Brier.Brier_uncertainty.attrs['name'] = 'uncertainty component of Brier score'
        
    return Brier


In [141]:
def compute_Brier_score_new(cmp_likelihood, ref_logical, indep_dims, cmp_prob=None):
    """ 
    Computes the Brier score(s) of an event given the comparison likelihood and reference logical 
    event data. When comparison probability bins are also provided, also computes the reliability, 
    resolution and uncertainty components of the Brier score
    """
    
    if isinstance(indep_dims, str):
        indep_dims = [indep_dims]
    
    ref_binary = ref_logical.copy()*1

    # Calculate total Brier score -----
    N = 1
    if indep_dims == None:
        Brier = (1 / N) * ((cmp_likelihood - ref_binary) ** 2).sum(dim=indep_dims).rename('Brier_score')
    else: 
        N = [N * len(cmp_likelihood[indep_dim]) for indep_dim in indep_dims][0]
        Brier = (1 / N) * ((cmp_likelihood - ref_binary) ** 2).rename('Brier_score')
        
    # Calculate components
    if cmp_prob is not None:

        # Initialise probability bins -----
        cmp_prob_edges = utils.get_bin_edges(cmp_prob)

        # Initialise mean_cmp_likelihood array -----
        mean_cmp_likelihood = cmp_likelihood.copy(deep=True)
        
        # Loop over probability bins -----
        mean_cmp_prob_list = []
        cmp_number_list = []
        ref_occur_list = []
        for idx in range(len(cmp_prob_edges)-1):
            # Logical of comparisons that fall within probability bin -----
            cmp_in_bin = (cmp_likelihood >= cmp_prob_edges[idx]) & \
                         (cmp_likelihood < cmp_prob_edges[idx+1])
            
            if indep_dims == None:
                # Mean comparison probability within current probability bin -----
                mean_cmp_prob_list.append(cmp_likelihood.where(cmp_in_bin,np.nan))
                
                # Number of comparisons that fall within probability bin -----
                cmp_number_list.append(cmp_in_bin)
                
                # Number of reference occurences where comparison likelihood is within probability bin -----
                ref_occur_list.append(((cmp_in_bin == True) & (ref_logical == True)))
            else:
                # Replace likelihood with mean likelihood (so that Brier components add to total) -----
                mean_cmp_likelihood = mean_cmp_likelihood.where(~cmp_in_bin).fillna( \
                                                         mean_cmp_likelihood.where(cmp_in_bin)
                                                                            .mean(dim=indep_dims))
                
                # Mean comparison probability within current probability bin -----
                mean_cmp_prob_list.append(cmp_likelihood.where(cmp_in_bin,np.nan) \
                                                        .mean(dim=indep_dims)) 
                
                # Number of comparisons that fall within probability bin -----
                cmp_number_list.append(cmp_in_bin.sum(dim=indep_dims))
                
                # Number of reference occurences where comparison likelihood is within probability bin -----
                ref_occur_list.append(((cmp_in_bin == True) & (ref_logical == True)) \
                                        .sum(dim=indep_dims)) 

        # Concatenate lists -----
        mean_cmp_prob = xr.concat(mean_cmp_prob_list, dim='comparison_probability')
        mean_cmp_prob['comparison_probability'] = cmp_prob
        cmp_number = xr.concat(cmp_number_list, dim='comparison_probability')
        cmp_number['comparison_probability'] = cmp_prob
        ref_occur = xr.concat(ref_occur_list, dim='comparison_probability')
        ref_occur['comparison_probability'] = cmp_prob

        # Compute Brier components -----
        base_rate = ref_occur / cmp_number
        Brier_reliability = (1 / N) * (cmp_number*(mean_cmp_prob - base_rate) ** 2) \
                                       .sum(dim='comparison_probability',skipna=True)
        if indep_dims == None:
            sample_clim = ref_binary
        else:
            sample_clim = ref_binary.mean(dim=indep_dims)
        Brier_resolution = (1 / N) * (cmp_number*(base_rate - sample_clim) ** 2) \
                                      .sum(dim='comparison_probability',skipna=True)
        Brier_uncertainty = sample_clim * (1 - sample_clim)
        
        # When a binned approach is used, compute total Brier using binned probabilities -----
        # (This way Brier_total = Brier_reliability - Brier_resolution + Brier_uncertainty)
        if indep_dims == None:
            Brier_total = (1 / N) * ((mean_cmp_likelihood - ref_binary) ** 2)
        else:
            Brier_total = (1 / N) * ((mean_cmp_likelihood - ref_binary) ** 2).sum(dim=indep_dims)
        
        # Package in dataset -----
        Brier = Brier_total.to_dataset(name='Brier_total')
        Brier.Brier_total.attrs['name'] = 'total Brier score'
        Brier['Brier_reliability'] = Brier_reliability
        Brier.Brier_reliability.attrs['name'] = 'reliability component of Brier score'
        Brier['Brier_resolution'] = Brier_resolution
        Brier.Brier_resolution.attrs['name'] = 'resolution component of Brier score'
        Brier['Brier_uncertainty'] = Brier_uncertainty
        Brier.Brier_uncertainty.attrs['name'] = 'uncertainty component of Brier score'
        
    return Brier

In [142]:
event = '(> 100) and (< 600)'

# Compute the event data for forecast likelihood and observations -----
fcst_likelihood = skill.compute_likelihood(skill.did_event(da_fcst, event))
obsv_logical = skill.did_event(da_obsv, event)

# Compute the Brier score -----
fcst_probabilities = np.linspace(0,1,len(da_fcst['ensemble'])-6)

In [143]:
Brier_old = compute_Brier_score_old(fcst_likelihood,obsv_logical,cmp_prob=fcst_probabilities,
                                      indep_dims='init_date')

Brier_old

<xarray.Dataset>
Dimensions:            (lead_time: 24)
Coordinates:
  * lead_time          (lead_time) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 ...
Data variables:
    Brier_total        (lead_time) float64 0.2488 0.2182 0.1521 0.1328 ...
    Brier_reliability  (lead_time) float64 0.2488 0.2182 0.05207 0.1328 ...
    Brier_resolution   (lead_time) float64 0.0 0.16 0.14 0.24 0.01 0.16 0.14 ...
    Brier_uncertainty  (lead_time) float64 0.0 0.16 0.24 0.24 0.16 0.16 0.24 ...

In [144]:
Brier_new = compute_Brier_score_new(fcst_likelihood,obsv_logical,cmp_prob=fcst_probabilities,
                                      indep_dims='init_date')

Brier_new

<xarray.Dataset>
Dimensions:            (lead_time: 24)
Coordinates:
  * lead_time          (lead_time) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 ...
Data variables:
    Brier_total        (lead_time) float64 0.2488 0.2182 0.1521 0.1328 ...
    Brier_reliability  (lead_time) float64 0.2488 0.2182 0.05207 0.1328 ...
    Brier_resolution   (lead_time) float64 0.0 0.16 0.14 0.24 0.01 0.16 0.14 ...
    Brier_uncertainty  (lead_time) float64 0.0 0.16 0.24 0.24 0.16 0.16 0.24 ...

In [145]:
Brier_old.equals(Brier_new)

True